## k_fold.ipynb

### Valutazione di modelli con grid search e random search tramite K-fold cross validation
> Questo notebook fornisce un ambiente interattivo per l'esplorazione e l'addestramento di reti neurali per il riconoscimento delle cifre del dataset MNIST, utilizzando la libreria custom messa a disposizione dagli autori. Inoltre, fornisce un framework completo per eseguire la k-fold cross validation e la selezione degli iper-parametri.

Autori:
- Alessandro Trincone
- Mario Gabriele Carofano

### Importazione di moduli e librerie
> Per il corretto funzionamento del notebook, è necessario importare alcuni moduli e librerie, tra cui:
> - la libreria `numpy` per l'utilizzo di strutture dati e funzioni matematiche efficienti;
> - il modulo `NeuralNetwork` per costruire reti neurali tramite la libreria custom messa a disposizione dagli autori.

In [ ]:
import auxfunc
import constants
from artificial_neural_network import NeuralNetwork
from training_params import TrainingParams
import dataset_functions as df
import plot_functions as pf

In [ ]:
import numpy as np
import pprint
import time
from datetime import datetime
import copy
import gc

In [ ]:
import pandas as pd
import random
import os

Con questo quaderno, gli utenti possono:

### Eseguire la K-fold cross validation
> Questa tecnica di validazione può essere utilizzata per la selezione degli iper-parametri di addestramento come:
> - `eta_minus` e `eta_plus`, specifici dell'algoritmo di aggiornamento dei pesi RPROP;
> - `l_sizes`, cioè la lista delle dimensioni dei layer interni della rete neurale.

In [ ]:
def k_fold_cross_validation(
        out_directory : str,
        Xtrain : list[np.ndarray],
        Ytrain : list[np.ndarray],
        k : int = constants.DEFAULT_K_FOLD_VALUE,
        l_sizes : list[int] = constants.DEFAULT_LAYER_NEURONS,
        params : TrainingParams = None
) -> list[dict]:
    
    """
        E' una tecnica di validazione che utilizza una parte indipendente del training set per la fase di validazione. Si utilizza per la selezione degli iper-parametri che restituiscono il minor errore di validazione sull'addestramento del modello.

        Parameters:
        -   out_directory : la directory di output dove salvare i grafici della k-fold cross validation.
        -   Xtrain : la matrice di esempi da classificare.
        -   Ytrain : la matrice di etichette corrispondenti agli esempi (ground truth).
        -   k : e' un numero intero che indica in quante fold dividere il training set.
        -   l_sizes : e' una lista contenente le dimensioni di uno o piu' hidden layer e dell'output layer della rete neurale.
        -   params : e' un oggetto della classe TrainingParams che contiene alcuni iper-parametri per la fase di addestramento della rete neurale.

        Returns:
        -   k_fold_report : una lista di metriche di valutazione relative ai valori di errore e di accuracy di validazione ottenuti durante le fasi di addestramento dei modelli sulle diverse fold.
    """

    k_fold_report = []

    Xfolds, Yfolds = df.split_dataset(Xtrain, Ytrain, k)

    for i in range(k):

        print(f"\nFold {i+1} di {k}")

        # Tutti gli altri parametri di NeuralNetwork sono inizializzati con i valori default
        net = NeuralNetwork(l_sizes=l_sizes, t_par=params)

        training_fold = np.concatenate([fold for j, fold in enumerate(Xfolds) if j != i])
        training_labels = np.concatenate([fold for j, fold in enumerate(Yfolds) if j != i])
        validation_fold = Xfolds[i]
        validation_labels = Yfolds[i]

        history_report = net.train(
            training_fold, training_labels,
            validation_fold, validation_labels
        )

        train_errs  = [r.training_error for r in history_report]
        val_errs    = [r.validation_error for r in history_report]

        train_accs  = [r.training_accuracy for r in history_report]
        val_accs    = [r.validation_accuracy for r in history_report]

        k_fold_report.append({
            "Fold"          : i+1,
            "ET_history"    : train_errs,
            "EV_history"    : val_errs,
            "E_value"       : net.training_report.validation_error,
            "E_min"         : np.min(val_errs),
            "E_max"         : np.max(val_errs),
            "AT_history"    : train_accs,
            "AV_history"    : val_accs,
            "A_value"       : net.training_report.validation_accuracy,
            "A_min"         : np.min(val_accs),
            "A_max"         : np.max(val_accs)
        })

        net.save_network_to_file(out_directory, out_name=f"net_Fold{i+1}.pkl")

        del net
        del training_fold, training_labels
        del validation_fold, validation_labels
        gc.collect()

        if constants.DEBUG_MODE:
            break

    # end for i
    
    return k_fold_report

# end

### Eseguire la grid search per il tuning degli iper-parametri

In [ ]:
def grid_search_cv() -> list[tuple[float, float, int]]:
    
    """
        Ricerca la miglior combinazione di iper-parametri in uno spazio dei parametri ben delimitato da una griglia di valori (da qui, il nome Grid Search).

        Returns:
        -   Restituisce una lista di iper-parametri (cioe': eta minus, eta plus e il numero di neuroni per l'hidden layer) campionati da uno spazio dei parametri limitato in una griglia di valori.
    """

    # Il valore tipico per "eta minus" e' compreso tra 0.5 e 0.9.
    # Valori piccoli riducono considerevolmente il passo di aggiornamento migliorando la stabilita', ma potenzialmente rallentando la convergenza. Al contrario, valori piu' grandi consentono una convergenza piu' rapida ma con il rischio di instabilita'.
    eta_minus_values = [0.5, 0.7, 0.9]

    # Il valore tipico per "eta plus" e' compreso tra 1.2 e 1.5.
    # Si possono utilizzare valori piu' o meno piccoli in base alle necessita' della propria applicazione, e i vantaggi / svantaggi di questa scelta sono gli stessi di cui sopra.
    eta_plus_values = [1.2, 1.3, 1.5]

    # Per problemi di classificazione (es. riconoscimento di cifre scritte a mano), un intervallo comune per il numero di neuroni e' da 10 a 100 per uno o due hidden layer.
    # Tuttavia, siccome l'input ha alta dimensionalita', potrebbe essere necessario aumentare il numero di neuroni per catturare le caratteristiche importanti. Ad esempio, per immagini 28x28, l'hidden layer puo' avere un numero di neuroni compreso tra 32 e 128.
    hidden_layer_values = [32, 64, 128]

    # Si restituiscono tutte le possibili combinazioni di iper-parametri in questo spazio limitato.
    return [
        (x, y, z)
        for x in eta_minus_values
        for y in eta_plus_values
        for z in hidden_layer_values
    ]

# end

# RIFERIMENTI
# Grid Search vs. Random Search : https://www.youtube.com/watch?v=G-fXV-o9QV8

### Eseguire la random search per il tuning degli iper-parametri

In [ ]:
def random_search_cv(
        r : int = constants.DEFAULT_RANDOM_COMBINATIONS
) -> list[tuple[float, float, int]]:
    
    """
        Ricerca la miglior combinazione di iper-parametri campionando in modo casuale (da qui, il nome Random Search) all'interno dell'intero spazio dei parametri un numero limitato di combinazioni.

        Parameters:
        -   r : e' un numero intero che indica il numero di combinazioni di iper-parametri da campionare.

        Returns:
        -   Restituisce una lista di iper-parametri (cioe': eta minus, eta plus e il numero di neuroni per l'hidden layer) campionati in modo casuale dall'intero spazio.
    """

    return [(
        np.random.uniform(low=0.5, high=0.9),
        np.random.uniform(low=1.2, high=1.5),
        np.random.randint(low=32, high=128)
    ) for _ in range(r)]

# end

### Valutare le prestazioni e le predizioni della rete neurale
> Grazie alla libreria messa a disposizione dagli autori del notebook, è possibile addestrare e valutare le prestazioni (nonché le predizioni) della rete neurale addestrata tramite specifiche funzionalità per la visualizzazione ed il salvataggio di report nella forma di tabelle, grafici di diverso tipo e anche immagini.

> In questo esempio di utilizzo:
> - Si carica il MNIST training set e il MNIST test set in due strutture dati separate.
> - Si memorizza in 'out_directory' la directory di output dove salvare i grafici richiesti.
> - Si applica la Grid Search K-Fold Cross Validation separando il MNIST training set in due sottoinsiemi distinti e separati (training e validation)
> - Si applica la Random Search K-Fold Cross Validation (applicando la suddivisione come sopra).
> - Si addestra una nuova rete neurale sulla miglior combinazione di iper-parametri ottenuta sull'intero MNIST training set.
> - Si verificano le predizioni della rete neurale sul MNIST test set.

In [ ]:
idTrain, Xtrain, Ytrain, idTest, Xtest, Ytest = df.loadDataset(constants.COPPIE_TRAINING, constants.COPPIE_TEST)

In [ ]:
out_directory = constants.OUTPUT_DIRECTORY + datetime.now().strftime(constants.OUTPUT_DATE_TIME_FORMAT) + "/"

In [ ]:
for search_type in ["Grid Search", "Random Search"]:

    print(f"\nValutazione con {search_type} iniziata: {datetime.now().strftime(constants.PRINT_DATE_TIME_FORMAT)}")
    start_time = time.time()

    # Si inizializza un "DataFrame" per la raccolta delle metriche di valutazione.
    stats = pd.DataFrame({
        "Eta minus"     : [],
        "Eta plus"      : [],
        "Hidden layer"  : [],
        "Mean error"    : [],
        "Std error"     : [],
        "Mean accuracy" : [],
        "Std accuracy"  : [],
        "Elapsed time"  : []
    })

    # Selezione delle combinazioni di iper-parametri.
    params = grid_search_cv() if search_type == "Grid Search" else random_search_cv(constants.DEFAULT_RANDOM_COMBINATIONS)

    # Esecuzione della cross validation per ogni combinazione di iper-parametri.
    for i, p in enumerate(params):

        # Scelta della combinazione di iper-parametri corrente.
        em = p[0]
        ep = p[1]
        hl = p[2]

        print(f"\nCombinazione {i}:\n\tEta minus = {em},\n\tEta plus = {ep},\n\tHidden layer = {int(hl)}\n")
        k_fold_out = f"{out_directory}{search_type.replace(' ', '_').lower()}/"

        # Si applica la tecnica di validazione "K-fold cross validation" per valutare la combinazione di iper-parametri corrente.
        k_fold_report = k_fold_cross_validation(
            k_fold_out + f"comb_{i}/",
            Xtrain, Ytrain, constants.DEFAULT_K_FOLD_VALUE,
            [hl, 10],
            params=TrainingParams(
                eta_minus=em,
                eta_plus=ep
            )
        )

        # Si recuperano tutti i valori di errore e accuracy di validazione della fase di addestramento.
        fold_errs = [r['E_value'] for r in k_fold_report]
        fold_accs = [r['A_value'] for r in k_fold_report]

        end_time = time.time()
        tot_time = end_time - start_time

        # Si crea un dizionario per raccogliere tutte le metriche di valutazione riguardo la rete appena addestrata sulla combinazione di iper-parametri corrente.
        new_row = {
            "Eta minus"     : em,
            "Eta plus"      : ep,
            "Hidden layer"  : hl,
            "Mean error"    : np.mean(fold_errs),
            "Std error"     : np.std(fold_errs),
            "Mean accuracy" : np.mean(fold_accs),
            "Std accuracy"  : np.std(fold_accs),
            "Elapsed time"  : tot_time / 3600.0
        }

        # Si aggiunge il dizionario come nuova riga alla tabella generale.
        stats = pd.concat([stats, pd.DataFrame([new_row])], ignore_index=True)

        # Si ripristina l'indice per mantenere l'ordinamento cronologico.
        stats = stats.reset_index(drop=True)

        # Si disegna un'immagine contenente i risultati dell'esecuzione della K-fold.
        pf.plot_search_report(k_fold_out + f"comb_{i}/", search_type, k_fold_report, stats.iloc[-1])

        # Visualizzazione del tempo impiegato per la valutazione di una singola combinazione
        print(f"\nValutazione della combinazione {i} completata: {datetime.now().strftime(constants.PRINT_DATE_TIME_FORMAT)}")
        print(f"Tempo trascorso: {tot_time:.3f} secondi")

    # end for i

    # Si ordina il DataFrame in modo non decrescente per i valori della colonna 'Mean Error'.
    stats = stats.sort_values(by=['Mean error'])
    formatted_stats = stats.style.format({
        "Eta minus"     : '{:.5f}',
        "Eta plus"      : '{:.5f}',
        "Hidden layer"  : '{:.0f}',
        "Mean error"    : '{:.5f}',
        "Std error"     : '{:.5f}',
        "Mean accuracy" : '{:.2%}',
        "Std accuracy"  : '{:.2%}',
        "Elapsed time"  : '{:.5f}'
    })

    # Si visualizzano le statistiche raccolte in una tabella.
    display(formatted_stats)

    # Si salva il contenuto della tabella in un file .csv.
    os.makedirs(out_directory, exist_ok=True)
    formatted_stats.to_string(k_fold_out + 'stats.csv', delimiter=',')

    # Visualizzazione della miglior combinazione di iper-parametri.
    best_params_idx = stats['Mean error'].idxmin()
    print(f"\nMiglior combinazione:\n\tIndex = {best_params_idx},\n\tEta minus = {stats['Eta minus'][best_params_idx]:.5f},\n\tEta plus = {stats['Eta plus'][best_params_idx]:.5f},\n\tHidden layer = {stats['Hidden layer'][best_params_idx]:.0f}\n")

    # Visualizzazione del tempo impiegato per l'intera fase di valutazione.
    end_time = time.time()
    tot_time = end_time - start_time

    print(f"\nValutazione con {search_type} completata: {datetime.now().strftime(constants.PRINT_DATE_TIME_FORMAT)}")
    print(f"Tempo trascorso: {tot_time:.3f} secondi")

# RIFERIMENTI
# https://saturncloud.io/blog/how-to-insert-a-row-to-pandas-dataframe/
# https://www.geeksforgeeks.org/display-the-pandas-dataframe-in-table-style/
# https://stackoverflow.com/questions/75956209/error-dataframe-object-has-no-attribute-append
# https://stackoverflow.com/questions/17006641/single-line-nested-for-loops
# https://stackoverflow.com/questions/20937538/how-to-display-pandas-dataframe-of-floats-using-a-format-string-for-columns

In [ ]:
# Recupero dei migliori iper-parametri tra grid search e random search.
# net_file = NeuralNetwork.load_network_from_file("...")
# print(net_file)
# ...

# Addestramento di una nuova rete neurale su questi iper-parametri.
# ...

In [ ]:
# Verifica delle prestazioni della rete neurale sul MNIST test set
# Salvataggio delle immagini / grafici sulle predizioni.
# net.test(
#     out_directory,
#     idTest, Xtest, Ytest,
#     plot_mode=constants.PlotTestingMode.ALL
# )

# net.predict(idTest, Xtest)